
# EDA DataSet ARQDIGITAL - People Analytics: Diagnóstico Preditivo de Turnover

**Visão Executiva da Análise**
Este documento apresenta a Análise Exploratória de Dados (EDA) que fundamenta a construção do nosso **Modelo Preditivo de Pedidos de Demissão (Turnover Voluntário)**

O objetivo dessa análise é realizar uma busca aprofundada e completa na qualidade dos dados da nossa *One Big Table* (OBT) e descobrir quais os verdadeiros motivos que estão influenciando a reteção. Para isso, cruzamos quatro pontos estratégicos da nossa operação:

* **Fator Humano & Demográfico:** Idade, Gênero, Estado Cívil e Estrutura Familia (Dependentes).
* **Fator Organizacional:** Tempo de Casa, Nível Hierárquico, Tipo de Contaro e Departamento.
* **Fator Financeiro:** Salário Contratual, Valor Líquido e Pacote de Benefícios Corporativos.
* **Fator Cultural:** Mapeamento de Perfil Comportamental (Sólides Profiler).

**Regra de Negócio (Target)**

Nossa variável alvo (`target_pediu_demissao`) foi desenhada com uma trava de isolamento preciso no proprío DW Consolidado de RH. O alvo foca *exclusivamente* nas demissões voluntárias (1), contra a nossa base de talentos ativos (0). Demissões involuntárias foram retiradas para não contaminar o padrão do comportalmente de pedidos de demissão.

In [2]:
pip install scikit-learn

  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------  7.9/8.0 MB 56.3 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 49.6 MB/s  0:00:00
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   -------------- ------------------------- 13.4/36.3 MB 67.7 MB/s eta 0:00:01
   ------------------------ --------------- 22.5/36.3 MB 56.2 MB/s eta 0:00:01
   ----------------------------------- ---- 32.0/36.3 MB 52.5 MB/s eta 0:00:01
   ---------------------------------------- 36.3/36.3 MB 46.4 MB/s  0:00:00
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------

## Perguntas Inicias sobre o Nosso Dataframe

1. **O Volume (linhas e colunas).**

    - **Resposta**: 91 Linhas e 22 Colunas

2. Onde estão os Buracos (nulos).

    - **Resposta**: Nas colunas
    |Coluna| Quantidade|
    |-------|----------|
    |data_demissao| 64|
    |perfil_comportamental| 7|
    |salario_contratual| 4|
    |turno_trabalho| 4|
    |valor_liquido| 4|
    |cidade| 2|
    |estado| 2|
    |estado_civil| 1|
    |tipo_contrato| 1|
    |nivel_hierarquico| 1|


3. A nossa Proporção exata de quem saiu vs. quem ficou.

    - **Resposta**
| Target      | Proporçao |
|-------------|-----------|
| 0| 69.230769 |
| 1| 30.769231 |



In [4]:
# Marco 0 - Importando as libs de analise e visualização inicais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression


In [6]:
# Marco 1 EDA - Lendo o Dataset
path = r'C:\Users\joaop\AppData\Roaming\JetBrains\DataSpell2025.3\projects\workspace\base.csv'
df = pd.read_csv(path, sep = ',')
df

,colaborador_sk,data_nascimento,genero,estado_civil,escolaridade,cep,bairro,cidade,estado,data_admissao,...,departamento_nome_api,cargo_nome_api,turno_trabalho,tipo_contrato,total_beneficios_api,target_pediu_demissao,qtd_dependentes,perfil_comportamental,salario_contratual,valor_liquido
0,98,1988-08-23,Masculino,Casado,MBA,71917000,Norte (Águas Claras),Brasília,DF,2025-04-22,...,TI,COORDENADOR DE INFRAESTRUTURA E SEGURANÇA\t\t,Integral,CLT,2616.98,0,3,C,8654.00,4531.56
1,96,2004-09-18,Feminino,Solteiro,Superior (cursando),72220096,Ceilândia,Brasília,DF,2023-02-13,...,OPERACIONAL,ANALISTA OPERACIONAL I,Comercial,CLT,2225.52,0,0,PA,2491.00,2349.67
2,131,1982-09-17,Feminino,Casado,Pós Graduação,71907000,Norte (Águas Claras),Brasília,DF,2023-12-04,...,COMUNICAÇÃO,Analista de Comunicação e Marketing III,Comercial,CLT,0.00,1,2,PEC,6000.00,0.00
3,125,1979-11-06,Masculino,Divorciado,Superior Completo,70277060,Asa Sul,Brasília,DF,2025-09-22,...,NOVOS NEGÓCIOS,GERENTE DE NEGÓCIOS,Comercial,CLT,2829.92,0,1,E,15000.00,10672.68
4,93,2023-05-03,Masculino,Solteiro,Médio Completo,72583250,Setor Meireles (Santa Maria),Brasília,DF,2023-05-03,...,OPERACIONAL,ASSISTENTE DE OPERAÇÕES,Comercial,Temporário,0.00,1,0,EA,1964.83,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,37,2001-05-22,Feminino,Solteiro,Superior (cursando),72915286,Jardim dos Pinheiros,Águas Lindas de Goiás,GO,2023-07-10,...,OPERACIONAL,ANALISTA OPERACIONAL I,Comercial,CLT,2384.34,0,2,PA,2491.00,1820.35
87,46,1993-11-20,Masculino,Casado,Pós Graduação,71884561,Riacho Fundo II,NaN,NaN,2025-04-07,...,TI,ANALISTA DE SUPORTE DE SISTEMAS II,Comercial,CLT,2535.78,1,2,PC,4391.00,3973.76
88,39,1996-11-08,Feminino,Solteiro,Superior (cursando),66050140,Umarizal,Belém,PA,2023-01-09,...,OPERACIONAL,ASSISTENTE OPERACIONAL,Comercial,CLT,2362.77,0,0,CE,2413.00,1646.12
89,20,1994-12-27,Masculino,Solteiro,Superior Completo,70683190,Setor Noroeste,Brasília,DF,2018-12-17,...,TI,DIRETOR DE TECNOLOGIA,Comercial,CLT,2533.00,0,2,PA,15748.00,12267.28


In [7]:
# Marco 1.1 EDA - Entendo o dataset e suas caracteristicas
linhas, colunas = df.shape
print(f'O dataset tem um total de {linhas} linhas e {colunas} colunas')

O dataset tem um total de 91 linhas e 22 colunas


In [28]:
# Marco 1.2 EDA - Vendo a quantidade de Valores NUlos
df.isna().sum(axis = 0).sort_values(ascending = False)

data_demissao            64
perfil_comportamental     7
salario_contratual        4
turno_trabalho            4
valor_liquido             4
cidade                    2
estado                    2
estado_civil              1
tipo_contrato             1
nivel_hierarquico         1
colaborador_sk            0
escolaridade              0
genero                    0
data_nascimento           0
cep                       0
departamento_nome_api     0
bairro                    0
data_admissao             0
cargo_nome_api            0
total_beneficios_api      0
target_pediu_demissao     0
qtd_dependentes           0
dtype: int64

In [63]:
# Marco 1.3 EDA - Proporção de pedidos de demissão
df[['target_pediu_demissao']].value_counts(
    normalize=True
)

target_pediu_demissao
0                        0.692308
1                        0.307692
Name: proportion, dtype: float64

44.44444444444444
